In [61]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
# 로드될 때까지 기다리는 설정을 위한 import
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 상품 관련 내용을 크롤링하기 위해 import
from bs4 import BeautifulSoup
# 상품의 정보를 예쁘게 출력하기 위해 import
import json
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
# 엑셀 저장을 위해 추가
import os
import pandas as pd
from datetime import datetime

# 옵션 설정: Headless 모드로 실행되도록 설정 -> 창이 켜지지 않고 vscode 내에서 바로바로 실행 됨 250513 제리
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저 창 안 띄움
options.add_argument('--disable-gpu')  # GPU 가속 비활성화 (Windows 환경에서 필요할 수 있음)
options.add_argument('--no-sandbox')  # 리눅스에서 root 권한 문제 해결용
options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 제한 문제 해결용

# 크롬 드라이버 실행 (옵션 추가)
driver = webdriver.Chrome(options=options)

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.youtube.com/watch?v=w7uwgwj6aZM') # 침착맨 "버거킹, 우리의 아이디어를 꼭 들어주길 바랍니다"
# 유튜브 영상의 제목 html 요소가 로드될 때까지 최대 60초 대기(xpath 방식으로 구현)
base_info_results = {}

"""
영상 제목, 채널명, 구독자 수, 조회수, 업로드 날짜
"""
# XPath는 바뀔 수도 있으니, 에러가 뜬다면 html 다시 뜯으러 가야 함.
base_info = {
    "channel": '//*[@id="text"]/a',
    "title": '//*[@id="title"]/h1',
    "upload_date": '//*[@id="info"]/span[3]',
    "views": '//*[@id="info"]/span[1]',
    "subscriber": '//*[@id="owner-sub-count"]',
}

# base_info에 담아둔 key에 따른 value들을 xpath를 통해 돌아가면서 들고옴 (빈복 작업)
for bases, xpath in base_info.items():
    try:
        # 로딩될 때까지 최대 60초 기다림
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        # bases에 들어간 title, channel 등등의 내용(text)을 "base_info_results" 딕셔너리에 추가
        base_info_results[bases] = element.text
    # error 처리
    except Exception as e:
        print(f"Error fetching {bases}: {e}")
        base_info_results[bases] = "Not found"

# 유튜브 링크 컬럼 추가
base_info_results["youtube_link"] = 'https://www.youtube.com/watch?v=w7uwgwj6aZM'

# 컬럼명 매핑
column_name_mapping = {
    "channel": "채널명",
    "title": "영상 제목",
    "youtube_link": "영상 링크",
    "upload_date": "업로드일",
    "views": "조회수",
    "subscriber": "구독자 수",
}

# 결과 출력
print("\n=== Base Info ===")
for key, value in base_info_results.items():
    print(f"{column_name_mapping.get(key, key)}: {value}")

# 오늘 날짜 문자열 (YYMMDD)
today_str = datetime.today().strftime('%y%m%d')

# 기본 파일명 설정
base_filename = f'침착맨_유튜브_{today_str}'
ext = '.xlsx'
filename = f'{base_filename}{ext}'

# 중복 파일 확인 및 숫자 붙이기
counter = 1
while os.path.exists(filename):
    filename = f'{base_filename}_{counter}{ext}'
    counter += 1

# 원하는 컬럼 순서를 리스트로 지정
column_order = ['채널명', '영상 제목', '영상 링크', '업로드일', '조회수', '구독자 수']

# DataFrame 생성 및 컬럼명 변경
df = pd.DataFrame([base_info_results])
df = df.rename(columns=column_name_mapping)

# 원하는 컬럼 순서로 재정렬
df = df[column_order]

# 엑셀로 저장
df.to_excel(filename, index=False)
print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

# 드라이버 종료
try:
    # scraping logic here
    pass
finally:
    driver.quit()


=== Base Info ===
채널명: 침착맨
영상 제목: 버거킹, 우리의 아이디어를 꼭 들어주길 바랍니다
업로드일: 1일 전
조회수: 조회수 39만회
구독자 수: 구독자 281만명
영상 링크: https://www.youtube.com/watch?v=w7uwgwj6aZM

✅ 엑셀 파일로 저장 완료: 침착맨_유튜브_250513.xlsx
